<a href="https://colab.research.google.com/github/asis012/bert/blob/master/Basics_snorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basics Tutorial for Snorkel MeTaL

The purpose of this tutorial is to introduce the basic pipeline, classes, and debugging/analysis tools in Snorkel MeTaL. In this notebook, we'll investigate a single-task problem with synthetic data, with an emphasis on the basic design concepts of MeTaL. Check out the `tutorials/` directory to see how this basic pipeline extends to the full _multi-task_ setting, and other topics.

This tutorial consists of four steps, following the basic [data programming](https://arxiv.org/abs/1605.07723) pipeline as in [Snorkel](https://hazyresearch.github.io/snorkel/):
1. **Load Data:** In the _weakly supervised_ setting, we only have access to unlabeled data points `X`, matrix of noisy labels `L`, and dev/test labels `Y`*
2. **Train Label Model:** The purpose of the `LabelModel` is to estimate the unknown accuracies of the labeling functions, _without access to `Y`_, and then use this to denoise and combine them into a set of _probabilistic training labels_.
3. **Train End Model:** We can then use these training labels to supervise a discriminative classifier!
4. **Evaluate:** We evaluate this model on a held-out test set

Let's get started!

In [0]:
!pip install metal

     |████████████████████████████████| 143kB 5.0MB/s 
     |████████████████████████████████| 1.8MB 42.8MB/s 
     |████████████████████████████████| 51kB 22.8MB/s 
  Created wheel for metal: filename=metal-0.1.7-cp36-none-any.whl size=37567 sha256=9f0a7d108bb18d6322e74f5baa622de526b806c32ea92309a00c306a023a35bd
  Stored in directory: /root/.cache/pip/wheels/d6/c9/16/465b63d77648d0ccba3b6343ae82849b5f123151341f0f7687
  Created wheel for pyaws: filename=pyaws-0.2.33-cp36-none-any.whl size=54153 sha256=a33998062ea54f2cd6e2ef2c3cf958b28d68c3ffc3571c7e33f9b5cc081739cc
  Stored in directory: /root/.cache/pip/wheels/0f/6a/26/6f32da74d4be696d29a965f23ba1b6315ced1bde11d8b35de8
Successfully built metal pyaws
ERROR: awscli 1.16.209 has requirement botocore==1.12.199, but you'll have botocore 1.12.189 which is incompatible.
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0


In [16]:

!pip install git+https://github.com/HazyResearch/metal

  Cloning https://github.com/HazyResearch/metal to /tmp/pip-req-build-r2qw1fj7
  Running command git clone -q https://github.com/HazyResearch/metal /tmp/pip-req-build-r2qw1fj7
  Created wheel for snorkel-metal: filename=snorkel_metal-0.5.0-cp36-none-any.whl size=6482113 sha256=842068d47e7f04fe3cd583a7a3fce23afb725560619bd889ba56fbfc36321b2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-rg77kc6g/wheels/76/4e/05/2004ecfdd9320a5f0d0fa9288ab697939c388517a6df30f33d
Successfully built snorkel-metal


In [20]:
!pip install tensorboardX

     |████████████████████████████████| 225kB 5.0MB/s 


### Setup

We first need to make sure that the `metal/` directory is on our Python path. If the following cell runs without an error, you're all set. If not, make sure that you've installed `snorkel-metal` with pip (conda coming soon) or that you've added the repo to your path if you're running from source; for example, running `source add_to_path.sh` from the repository root.

In [0]:
import sys
sys.path.append('../../metal')
import metal

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Step 1: Load Data

The first step in a Snorkel MeTaL application is preparing your data.

### Concept 1: Required Data Types

In [2]:
from google.colab import files
uploaded = files.upload()

Saving basics_tutorial.pkl to basics_tutorial.pkl


In particular, MeTaL makes use of the following types of data (n = # examples, m = # labeling functions):
* X: an \[n\]-dim iterable of end model inputs (e.g., feature vectors or encoded sentences for an RNN)
* Y: an \[n\]-dim numpy.ndarray of target labels ($Y \in \{1,...,k\}^n$)
* L: an \[n,m\] scipy.sparse matrix of noisy labels ($L \in \{0,...,k\}^{n \times m}$, with label 0 reserved for abstentions

And optionally (for use with some debugging/analysis tools):
* D: an \[n\]-dim iterable of human-readable examples (e.g., sentences)

In this tutorial, we use pre-generated synthetic data, where each example is a bag of words drawn from a different gaussian distribution for each class from a 1000-word vocabulary, and our features are simply the 1000-dimensional vector of counts for each word. For some excellent resources on how to write labeling functions, see the Snorkel tutorials at https://github.com/HazyResearch/snorkel/tree/master/tutorials.

In [0]:
import pickle

with open("basics_tutorial.pkl", 'rb') as f:
    X, Y, L, D = pickle.load(f)

In [7]:
print(X.shape)
print(Y.shape)
print(L.shape)

torch.Size([10000, 1000])
(10000,)
(10000, 10)


If you need to divide your data into splits, you can do so with the provided utility function. We split our data 80/10/10 into train/dev/test, stratifying by the labels in `Y` to ensure a similar class balance in each split.

In [39]:
from metal.utils import split_data

Xs, Ys, Ls, Ds = split_data(X, Y, L, D, splits=[0.8, 0.1, 0.1], stratify_by=Y, seed=123)

print(Ls[1])
print(Ys[1])

  (0, 0)	2
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	2
  (0, 8)	2
  (0, 9)	2
  (1, 0)	2
  (1, 3)	2
  (1, 4)	1
  (1, 6)	1
  (1, 8)	2
  (1, 9)	2
  (2, 0)	2
  (2, 1)	1
  (2, 2)	2
  (2, 3)	2
  (2, 4)	2
  (2, 5)	1
  (2, 6)	1
  (2, 7)	2
  (2, 8)	2
  (2, 9)	2
  (3, 0)	1
  :	:
  (997, 0)	2
  (997, 1)	2
  (997, 2)	2
  (997, 3)	1
  (997, 4)	1
  (997, 5)	1
  (997, 6)	1
  (997, 9)	1
  (998, 0)	2
  (998, 1)	1
  (998, 2)	1
  (998, 3)	1
  (998, 4)	2
  (998, 5)	2
  (998, 6)	1
  (998, 7)	1
  (998, 8)	1
  (998, 9)	2
  (999, 0)	1
  (999, 2)	1
  (999, 3)	1
  (999, 4)	2
  (999, 5)	2
  (999, 8)	1
  (999, 9)	1
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 

### Investigate Label Matrices

asdfghjk

MeTaL comes with a number of analysis tools for investigating your label matrix. 
For example, you can view summary statistics for each labeling function.
If you have a dev set with gold labels, you can view the empirical accuracy of your functions, in addition to the core summary statistics. If you do not have a dev set with gold labels, simply do not specify optional argument Y.

Polarity is a list of classes that a labeling function (LF) predicts for one or more examples. Coverage is the fraction of examples that the labeling function predicts over (as opposed to labeling 0 to abstain). Overlaps is the fraction of examples that the labeling function predicts over that at least one other LF predicts over as well. Conflicts is the fraction of examples that the LF predicts as one class but at least one other LF predicts that same example as a different class. Correct, Incorrect, and Empirical Accuracy are all comparisons to the gold standard labels and thus omitted if Y is not specified.

In [19]:
from metal.analysis import lf_summary

lf_summary(Ls[1],Y=Ys[1])

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,"[1, 2]",0.700,0.700,0.655,374,326,0.534286
1,"[1, 2]",0.735,0.735,0.700,468,267,0.636735
2,"[1, 2]",0.744,0.744,0.701,500,244,0.672043
3,"[1, 2]",0.748,0.748,0.706,540,208,0.721925
4,"[1, 2]",0.719,0.719,0.675,454,265,0.631433
5,"[1, 2]",0.746,0.746,0.696,457,289,0.612601
6,"[1, 2]",0.762,0.762,0.728,509,253,0.667979
7,"[1, 2]",0.796,0.796,0.744,593,203,0.744975
8,"[1, 2]",0.811,0.811,0.756,642,169,0.791615
9,"[1, 2]",0.784,0.784,0.735,559,225,0.713010


If you're interested in more graphical investigative tools, you can take a look at the Visualization tutorial.  
Once you're satisfied with your label matrices, it's time to aggregate the labels using the label model and train an end model with the resulting probabilistic labels.

## Step 2: Train Label Model

### Concept: Config dicts

All Snorkel MeTaL models are driven by configuration dictionaries, or config dicts; that is, all settings for running the given model are stored in a single dictionary, `self.config`. 

When a model is initialized or the train_model() method is called, any extra keyword arguments passed by the user will be used to update the config dict. Then, throughout the class's methods, whenever a setting needs to be looked up, it is pulled from this dictionary, rather than required as a keyword. This has a number of benefits:

1. Default values are stored in only one place, the default config dict, rather than in multiple method signatures.
1. Code maintenance is simplified, since fewer keyword arguments need to be passed up and down the call stack.
2. Logging is simplified, since the complete settings for a given model can be logged by simply writing the config dict (a python dictionary) to file.

### Label Model

When constructing a LabelModel, the only required argument is `k`, the cardinality of the task. For example, k=2 means a binary classification task where 1 and 2 are the class labels, and 0 is reserved for an abstain.

In [0]:
from metal.label_model import LabelModel
label_model = LabelModel(k=2, seed=123)

The only required argument to `LabelModel.train_model()` is a label matrix. All other keyword arguments are optional. 

For example, if you know the class_balance of your data, you can pass that in as a list or array; alternatively, you may pass in the target labels of your dev set (`Y_dev`) and have the class balance estimated from that. If neither of these is provided, the classes are assumed to be uniformly distributed.

Any other keyword arguments (e.g., the number of epochs (`n_epochs`), logging frequency (`log_train_every`), learning rate (`lr`), L2 regularization (`l2`), optimizer type (`optimizer`), etc.) will be used to update the config dict.

In [0]:
%%time
label_model.train_model(Ls[0], Y_dev=Ys[1], n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.039]
[100 epo]: TRAIN:[loss=0.003]
[150 epo]: TRAIN:[loss=0.002]
[200 epo]: TRAIN:[loss=0.002]
[250 epo]: TRAIN:[loss=0.002]
[300 epo]: TRAIN:[loss=0.002]
[350 epo]: TRAIN:[loss=0.002]
[400 epo]: TRAIN:[loss=0.002]
[450 epo]: TRAIN:[loss=0.002]
[500 epo]: TRAIN:[loss=0.002]
Finished Training
CPU times: user 993 ms, sys: 51.8 ms, total: 1.05 s
Wall time: 516 ms


You can test the quality of our label model on our dev set as a sanity check, but we'll see if we can do better in Step 3 by using the predictions of the label model to train a discriminative model over a larger feature set than just the outputs of these ten labeling functions.

In [0]:
score = label_model.score((Ls[1], Ys[1]))

Accuracy: 0.879
        y=1    y=2   
 l=1    181    65    
 l=2    56     698   


We can see that our trained `LabelModel` outperforms the baseline of taking the majority vote label by approximately 4 accuracy points.

In [0]:
print('Trained Label Model Metrics:')
scores = label_model.score((Ls[1], Ys[1]), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.879
Precision: 0.764
Recall: 0.736
F1: 0.749
        y=1    y=2   
 l=1    181    65    
 l=2    56     698   


In [0]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(seed=123)
print('Majority Label Voter Metrics:')
scores = mv.score((Ls[1], Ys[1]), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.836
Precision: 0.623
Recall: 0.841
F1: 0.716
        y=1    y=2   
 l=1    207    39    
 l=2    125    629   


We can see that our trained `LabelModel` outperforms the baseline of taking the majority vote label by approximately 0.04 in accuracy and 0.03 in F1 on the dev set. However, it has lower recall. Which of these metrics matters most will vary by application.

### Concept: Classifier Base Class

Both the `LabelModel` and `EndModel` in MeTaL are children of the parent `Classifier` class. The `Classifier` class defines three important methods related to making predictions:
* `predict_proba()`: Returns an [n, k+1] numpy array of probabilistic labels (k+1 because of the option to abstain)
* `predict()`: Returns an [n]-dim numpy array of  int labels in {0,...,k+1}
* `score()`: Returns the score (default: accuracy) of the predictions with respect to target labels

Following the data programming paradigm, we will use our label model's probabilistic (float) predictions as training labels for a discriminative end model. Because we want probabilistic labels, we use the `predict_proba()` method here. The predictions are in the form of an [n,k] np.ndarray of floats summing to 1.0 in each row, where `Y_ps[i,j]` is the predicted probability of the $i$th data point having true label $j$.

In [0]:
# Y_train_ps stands for "Y[labels]_train[split]_p[redicted]s[oft]"
Y_train_ps = label_model.predict_proba(Ls[0])
Y_train_ps

array([[0.33879477, 0.66120523],
       [0.01750565, 0.98249435],
       [0.02757499, 0.97242501],
       ...,
       [0.74142135, 0.25857865],
       [0.98866597, 0.01133403],
       [0.38616857, 0.61383143]])

### Analysis tools

At this point, we may want to analyze our LabelModel's predictions so that we better understand what our model is learning.

We can take a look at the confusion matrix to get a sense for class size, error rates, and commonly confused classes.

In [0]:
from metal.analysis import confusion_matrix

Y_dev_p = label_model.predict(Ls[1])

cm = confusion_matrix(Ys[1], Y_dev_p)

        y=1    y=2   
 l=1    181    56    
 l=2    65     698   


If matplotlib is installed, we can also use some of the visualization tools provided in the `contrib/` directory to plot the label distributions.  
First, we observe that our problem has a severe class imbalance, which our classifier captures in a general sense.  
Second, we can look at how confident our model is in its predictions for class 1 and see that most have probabilities very close to 0.

## Step 3: Train End Model

We will now construct and train our `EndModel` on the predictions from the LabelModel. Here we describe the construction of a single-task end model. For details on the construction of multi-task end models, refer to the multi-task tutorial.

The `EndModel` consists of three components: an *input layer*, zero or more *middle layers*, and a *head layer*, with each layer consisting of a torch.nn.Module followed by various optional additional operators (e.g., a ReLU nonlinearity, batch normalization, and/or dropout). 

* **Input layer**:
The input module is an IdentityModule by default, which simply accepts arbitrary-length feature vectors from X as torch.Tensors and passes them on to the next module in the network. If, however, you would like provide your data in some other format, you may pass in a custom nn.Module which maps your input type to torch.Tensors (for example, if your data points are images, you may pass in a ResNet as your input module).

* **Middle layers**
The middle layers are layers between the input layer and head layer. Middle modules are nn.Linear by default.

* **Head layer**:
The head layer is the final layer, consisting of an nn.Linear module (plus a softmax operation when making predictions).
 
When initiated, the EndModel requires a list of output dimensions. 
The first element is the output dimension of the input module (which is equal to the dimensionality of your feature vectors if using the default Identitymodule).
The last element is the cardinality of the classifier.
The elements in the middle define the number and dimensionality of any middle layers.

In [0]:
from metal.end_model import EndModel
import torch
if torch.cuda.is_available():
    device = 'cuda'
else:
    device='cpu'
end_model = EndModel([1000,10,2], seed=123, device=device)


Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Sequential(
    (0): Linear(in_features=1000, out_features=10, bias=True)
    (1): ReLU()
  )
  (2): Linear(in_features=10, out_features=2, bias=True)
)



Once initiated, the network structure is printed so you can confirm that it captured the architecture you want.

To train the model, we pass in our unlabeled training data, our predicted probabilistic (float) labels from the label aggregator, a dev set X and Y if we want to checkpoint the best model seen so far as we train, and any other keyword arguments that you'd like to update in the config dict. For this synthetic problem, the trends in our data are fairly simple, so our model will begin to overfit very quickly. If we do begin to overfit and see our dev score go down, however, the best model seen at any epoch will be restored at the end of training.

In [0]:
end_model.train_model((Xs[0], Y_train_ps), valid_data=(Xs[1], Ys[1]), lr=0.01, l2=0.01, batch_size=256, 
                n_epochs=5, checkpoint_metric='accuracy', checkpoint_metric_mode='max')

[1 epo]: TRAIN:[loss=0.510] VALID:[accuracy=0.935]
Saving model at iteration 1 with best score 0.935
[2 epo]: TRAIN:[loss=0.477] VALID:[accuracy=0.973]
Saving model at iteration 2 with best score 0.973
[3 epo]: TRAIN:[loss=0.473] VALID:[accuracy=0.964]
[4 epo]: TRAIN:[loss=0.471] VALID:[accuracy=0.973]
[5 epo]: TRAIN:[loss=0.469] VALID:[accuracy=0.976]
Saving model at iteration 5 with best score 0.976
Restoring best model from iteration 5 with score 0.976
Finished Training
Accuracy: 0.976
        y=1    y=2   
 l=1    222    24    
 l=2     0     754   


## Step 4: Evaluate

With a trained EndModel, we can now evaluate performance on our held-out test set, observing a substantial boost in F1 score over using the LabelModel directly. Remember that the label model predicts Y from the labeling functions, L, whereas the end model predicts Y from the raw input, X.

In [0]:
print("Label Model:")
score = label_model.score((Ls[2], Ys[2]), metric=['accuracy','precision', 'recall', 'f1'])

print()

print("End Model:")
score = end_model.score((Xs[2], Ys[2]), metric=['accuracy','precision', 'recall', 'f1'])

Label Model:
Accuracy: 0.869
Precision: 0.747
Recall: 0.707
F1: 0.727
        y=1    y=2   
 l=1    174    72    
 l=2    59     695   

End Model:
Accuracy: 0.965
Precision: 1.000
Recall: 0.858
F1: 0.923
        y=1    y=2   
 l=1    211    35    
 l=2     0     754   


And there you go! If you'd like to learn about some of the other features in Snorkel MeTaL, including support for multi-task learning (MTL), hyperparameter tuning, and synthetic data generation, give some of the other tutorial notebooks a try!